In [1]:
import os
import pandas as pd
import altair as alt

In [2]:
os.listdir(os.path.join("..", "data", "playstation"))

['PS4_PS5.csv']

In [3]:
filename = "PS4_PS5.csv"

In [4]:
df = pd.read_csv(os.path.join("..", 'data', "playstation", filename))
df

,Product,Date,Qty
0,Playstation 4,FY2013,7.6
1,Playstation 4,FY2014,14.8
2,Playstation 4,FY2015,17.7
3,Playstation 4,FY2016,20.0
4,Playstation 4,FY2017,19.0
5,Playstation 4,FY2018,17.8
6,Playstation 4,FY2019,13.5
7,Playstation 4,FY2020,5.7
8,Playstation 4,FY2021,1.0
9,Playstation 5,FY2020,7.8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Product  13 non-null     object 
 1   Date     13 non-null     object 
 2   Qty      13 non-null     float64
dtypes: float64(1), object(2)
memory usage: 440.0+ bytes


In [6]:
df['dt'] = pd.to_datetime((df['Date'].str.replace('FY', '').astype(int) + 1).astype(str) + '-03-31')
df

,Product,Date,Qty,dt
0,Playstation 4,FY2013,7.6,2014-03-31
1,Playstation 4,FY2014,14.8,2015-03-31
2,Playstation 4,FY2015,17.7,2016-03-31
3,Playstation 4,FY2016,20.0,2017-03-31
4,Playstation 4,FY2017,19.0,2018-03-31
5,Playstation 4,FY2018,17.8,2019-03-31
6,Playstation 4,FY2019,13.5,2020-03-31
7,Playstation 4,FY2020,5.7,2021-03-31
8,Playstation 4,FY2021,1.0,2022-03-31
9,Playstation 5,FY2020,7.8,2021-03-31


In [7]:
df = df.sort_values(["Product", "dt"], ascending=[True, True])

In [8]:
fy_totals = df.groupby(["dt"])["Qty"].sum().reset_index()
fy_totals = fy_totals[fy_totals["dt"].dt.year < 2024]

df_models = df.copy()
df_models = df_models
df_models["GroupRank"] = df_models.groupby(["Product"])["dt"].rank(method="dense", ascending=True)

model_totals = df_models.groupby(["Product", "GroupRank", "dt"])["Qty"].sum().reset_index()
model_totals = model_totals[model_totals["dt"].dt.year < 2024]

In [9]:
model_totals_chart = alt.Chart(model_totals).mark_line(point={"shape": "square", "size": 50}).encode(
    x=alt.X('dt:T'),
    y=alt.Y('Qty:Q'),
    color=alt.Color('Product:N')
)

fiscal_year_totals_chart = alt.Chart(fy_totals).mark_line(color='black', strokeDash=[5, 5], point={"shape": "square", "size": 50, "color": "black"}).encode(
    x=alt.X('dt:T'),
    y=alt.Y('Qty:Q'),
)


chart = fiscal_year_totals_chart + model_totals_chart
chart.interactive()

alt.LayerChart(...)

In [10]:
chart.save("../figs/Playstation_Models_vs_Total.png")